In [86]:
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import importlib
import connect4
importlib.reload(connect4);
from connect4 import Connect4

In [94]:
#
# Hyper parameters
# 
alpha = 0.3
gamma = 0.9
epsilon = 0.3

In [95]:
#
# Create the model and optimizer
# 
class Connect4Cnn(nn.Module):
    def __init__(self):
        super(Connect4Cnn, self).__init__()
        self.conv1 = nn.Conv2d(3, 64,  kernel_size=3, padding=1)    
        #self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        #self.conv3 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.out = nn.Linear(64*6*7, 7)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        #x = F.relu(self.conv2(x))
        #x = F.relu(self.conv3(x))
        x = x.view(-1)
        x = self.out(x)
        return x

model = Connect4Cnn()
optimizer = torch.optim.SGD(model.parameters(), lr = alpha)
sum(p.numel() for p in model.parameters() if p.requires_grad)

20615

In [96]:
#
# Load model from checkpoint
#
games = 0
if games > 0:
    cp = torch.load(f'connect4-{games}.nn');
    model.load_state_dict(cp['model_state_dict']);
    optimizer.load_state_dict(cp['optimizer_state_dict']);

In [97]:
@torch.no_grad()
def checkpoint(step):
    train = model.training
    if train: model.eval();
    print(f"{step}: checkpoint...")
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, f'connect4-{step}.nn');

    dummy_input = Connect4().state
    torch.onnx.export(model, dummy_input, f"connect4-{step}.onnx");

    print(f"{step}: checkpoint saved.")
    if train: model.train();


In [98]:
#
# Validation
#
@torch.no_grad()
def validate(numberOfGames):
    print('Validation...')
    xw, xd, xl, ow, od, ol  = connect4.validate(model, numberOfGames)
    xtotal = xw + xd + xl
    ototal = ow + od + ol
    print(f"Cross: {100*xw/xtotal:.2f}% of {xtotal} ({xw}/{xd}/{xl})")
    print(f"Circle: {100*(ow+od)/ototal:.2f}% of {ototal} ({ow}/{od}/{ol})")

In [93]:
#
# TRAINING
#
log_interval = 10000
validation_interval = 50000
validation_games = 20000
checkpoint_interval = 50000
losses = []
model.train()
for _ in range(100000):
    env = Connect4()
    done = False
    games += 1
    loss = 0
    moves = 0

    while not done:
        q = model(env.state)
        targetq = q.detach().clone()

        e = random.uniform(0, 1)
        if e < epsilon:
            action = random.choice([a for a in range(7) if env.is_valid(a)])
        else:
            action = max([a for a in range(7) if env.is_valid(a)], key=lambda x: q[x])
        
        env.move(action)

        if env.winner != 0:
            targetq[action] = 1
            done = True
        elif env.full:
            targetq[action] = 0 #if env.player == 2 else 0.1
            done = True
        else:
            model.eval()
            with torch.no_grad():
                next_q = model(env.state)
                next_max = -max([next_q[a] for a in range(7) if env.is_valid(a)])
                reward = -0.1 #if env.player == 2 else 0
                targetq[action] = reward + gamma * next_max
            model.train()
            
        loss += F.mse_loss(q, targetq)
        moves += 1

    loss /= moves
    losses.append(loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if games % log_interval == 0:
        print(f'{games}: average loss: {sum(losses)/len(losses)}')
        losses = []
    if games % validation_interval == 0:
        validate(validation_games)
    if games % checkpoint_interval == 0:
        checkpoint(games)


10000: average loss: 0.008709470394614617
20000: average loss: 0.007954917913966346


KeyboardInterrupt: 

### Connect4 CNN training

| Games     | Loss                  | Cross     | Circle
| :-------: | :-----------:         | :-------: | :-------: 
| 0         | -                     | 34.77%    | 30.11%
| 10000     | 0.008709470394614617  | -         | -
| 20000     | 0.007954917913966346  | -         | -


In [ ]:
Validate